# Rede GRU
## Fine Tuning

In [1]:
import pandas as pd
import numpy as np
from keras.utils import *

labels = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/activity_labels.txt", header=None, sep="\s+")

X_train = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt", header=None, sep="\s+")
y_train = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/train/Y_train.txt", header=None)

X_test = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt", header=None, sep="\s+")
y_test = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt", header=None)

X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((7352, 1, 561), (7352, 7), (2947, 1, 561), (2947, 7))

In [2]:
from keras.models import * 
from keras.layers import *
from keras.optimizers import *

def build_model(hp):
    modelo = Sequential()
    modelo.add(GRU(hp.Int("units_gru_1", min_value=1, max_value=600), return_sequences=True))
    modelo.add(GRU(hp.Int("units_gru_2", min_value=1, max_value=600)))
    modelo.add(BatchNormalization())
    modelo.add(Dense(7, activation="softmax"))
    modelo.compile(loss=hp.Choice("loss", ['binary_crossentropy', 'sparse_categorical_crossentropy']), optimizer=Adam(learning_rate=hp.Choice("learning_rate", [0.0001, 0.001, 0.01])), metrics=["accuracy"])
    return modelo

In [3]:
from keras_tuner import *
from keras.callbacks import *

tuner = BayesianOptimization(hypermodel=build_model, objective='accuracy')
tuner.search(X_train, y_train, validation_data=(X_test, y_test), batch_size=10, epochs=20)

Trial 10 Complete [00h 00m 01s]

Best accuracy So Far: 0.9834058880805969
Total elapsed time: 00h 16m 17s


In [4]:
hyperparameters = tuner.get_best_hyperparameters()[0]
print(f"Unidades GRU 1: {hyperparameters.get('units_gru_1')}")
print(f"Unidades GRU 2: {hyperparameters.get('units_gru_2')}")
print(f"Função de Perda: {hyperparameters.get('loss')}")
print(f"Learning Rate: {hyperparameters.get('learning_rate')}")

Unidades GRU 1: 268
Unidades GRU 2: 286
Função de Perda: binary_crossentropy
Learning Rate: 0.0001


In [5]:
best_model = tuner.get_best_models()[0]
best_model.summary()

C:\Users\daniel.a.goncalves\Desktop\tedi\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 1, 268)              │         668,124 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 286)                 │         477,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 286)                 │           1,144 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 7)                   │           2,009 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,148,325 (4.38 MB)

 Trainable params: 1,147,753 (4.38 MB)

 Non-trainable params: 572 (2.23 KB)